In [1]:
from concurrent.futures import ThreadPoolExecutor

import geopandas as gpd
import numpy as np
import odc.geo  # noqa: F401
# from dea_tools.classification import predict_xr
from odc.stac import load
from pystac_client import Client
from shapely import geometry
from sklearn.ensemble import RandomForestClassifier


In [2]:
fiji_bbox = [177.2, -18.4, 178.9, -17.2]
fiji_bbox_geometry = geometry.box(*fiji_bbox)
fiji_bbox_geometry

# training_file = "training_data/lulc_bare_land.geojson"
training_file = "lulc_combined.geojson"

tdata = gpd.read_file(training_file, bbox=fiji_bbox_geometry)
tdata.explore()

In [3]:
collection = "dep_ls_geomad"

client = Client.open("https://stac.staging.digitalearthpacific.org")

items = list(client.search(
    collections=[collection],
    bbox=fiji_bbox,
    datetime="2023"
).items())

print(f"Found {len(items)} items")

Found 6 items


## Note to future Nick

Wait for Alex to produce S-2 GeoMAD, then do the real work with that.

Between now and then, experiment with doing another LOAD step, like we're doing here, but with S-1.

So do the following:

* Search the MSPC STAC API for S-1 Items over our BBOX area over the year of interest
* Load them using odc.stac.load and do an annual summary, probably using a mean over time
* Maybe compute stdev too
* then add a HH/HV too, so you'll have a single timestep with HH_Mean, HV_Mean, HH_StdDev, HV_StdDev and HH/HV
* That can be added to the below data from GeoMAD.

Hint, when you use `load`, you can pass in the existin data. So first load GeoMad, then get `load(items, geobox=data.geobox)` 


In [25]:
bands = [
    "red",
    "green",
    "blue",
    "nir08",
    "swir16",
    "swir22",
    "emad",
    "bcmad",
    "smad",
]

data = load(
    items,
    measurements=bands,
    chunks={"x": 100, "y": 100, "time": 1},
)

data = data.squeeze("time")

data

<xarray.Dataset>
Dimensions:      (y: 6401, x: 9600)
Coordinates:
  * y            (y) float64 -1.888e+06 -1.888e+06 ... -2.08e+06 -2.08e+06
  * x            (x) float64 2.952e+06 2.952e+06 2.952e+06 ... 3.24e+06 3.24e+06
    spatial_ref  int32 3832
    time         datetime64[ns] 2023-01-01
Data variables:
    red          (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    green        (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    blue         (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    nir08        (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    swir16       (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    swir22       (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    emad         (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    bcmad        (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    smad         (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>

In [5]:
# Get S-1 dependencies

from planetary_computer import sign_url
import os

In [6]:
# Get the URL for STAC API on MSPC
catalogue_s1 = "https://planetarycomputer.microsoft.com/api/stac/v1/"

collection_s1 = "sentinel-1-rtc"

client_s1 = Client.open(catalogue_s1)

In [7]:
# Search for datasets

fiji_bbox = [177.2, -18.4, 178.9, -17.2]

datetime = "2023"


In [10]:
# Get a pystac client for the MSPC
items_s1 = list(client_s1.search(collections=[collection_s1], bbox=fiji_bbox, datetime=datetime).items())
print(len(items_s1))

133


In [11]:
# Authorised access to MPC data
os.environ["PC_SDK_SUBSCRIPTION_KEY"] = "84162f5502174b1b838239e74a44898d"

In [16]:
# Load data

data_s1 = load(items_s1, chunks={}, groupby="solar_day", like=data, patch_url=sign_url)

data_s1 = data_s1.where(data_s1 != -32768)

data_s1

<xarray.Dataset>
Dimensions:      (time: 107, y: 6401, x: 9600)
Coordinates:
  * y            (y) float64 -1.888e+06 -1.888e+06 ... -2.08e+06 -2.08e+06
  * x            (x) float64 2.952e+06 2.952e+06 2.952e+06 ... 3.24e+06 3.24e+06
    spatial_ref  int32 3832
  * time         (time) datetime64[ns] 2023-01-01T06:40:42.198354 ... 2023-12...
Data variables:
    vh           (time, y, x) float32 dask.array<chunksize=(1, 6401, 9600), meta=np.ndarray>
    vv           (time, y, x) float32 dask.array<chunksize=(1, 6401, 9600), meta=np.ndarray>

In [ ]:
# Load data using the existing geomad geobox


In [17]:
# Create means over time
mean_s1 = data_s1.mean(dim="time")
mean_s1["vv_vh"] = mean_s1.vv/mean_s1.vh
mean_s1


<xarray.Dataset>
Dimensions:      (y: 6401, x: 9600)
Coordinates:
  * y            (y) float64 -1.888e+06 -1.888e+06 ... -2.08e+06 -2.08e+06
  * x            (x) float64 2.952e+06 2.952e+06 2.952e+06 ... 3.24e+06 3.24e+06
    spatial_ref  int32 3832
Data variables:
    vh           (y, x) float32 dask.array<chunksize=(6401, 9600), meta=np.ndarray>
    vv           (y, x) float32 dask.array<chunksize=(6401, 9600), meta=np.ndarray>
    vv_vh        (y, x) float32 dask.array<chunksize=(6401, 9600), meta=np.ndarray>

In [23]:
# Plot s1
# mean_s1.vv_vh.plot.imshow(size=8, robust=True)

In [26]:
# Merge datasets

merged = data.update(mean_s1)
merged

<xarray.Dataset>
Dimensions:      (y: 6401, x: 9600)
Coordinates:
  * y            (y) float64 -1.888e+06 -1.888e+06 ... -2.08e+06 -2.08e+06
  * x            (x) float64 2.952e+06 2.952e+06 2.952e+06 ... 3.24e+06 3.24e+06
    spatial_ref  int32 3832
    time         datetime64[ns] 2023-01-01
Data variables:
    red          (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    green        (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    blue         (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    nir08        (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    swir16       (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    swir22       (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    emad         (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    bcmad        (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    smad         (y, x) float32 dask.array<chunksize=(100, 100), meta=np.ndarray>
    vh           (y, x) float32 dask.array<chunksize=(6401, 9600), meta=np.ndarray>
    vv           (y, x) float32 dask.array<chunksize=(6401, 9600), meta=np.ndarray>
    vv_vh        (y, x) float32 dask.array<chunksize=(6401, 9600), meta=np.ndarray>

In [10]:
projected_training_data = tdata.to_crs("EPSG:3832")
training_array = []

def get_training_data(id_row):
    _, row = id_row
    cls = row["lulc_class"]
    # Add more classes as integers here.
    # Maybe use a look-up table.
    cls_id = 1 if cls == "bare_land" else 0
    id = row["id"]
    geom = row["geometry"]

    # Get xarray values at the point
    x = data.sel(x=geom.x, y=geom.y, method="nearest").compute()
    one_point = [id] + [cls_id] + [float(v) for v in x.values()]
    return one_point

with ThreadPoolExecutor(max_workers=10) as executor:
    training_array = list(executor.map(get_training_data, projected_training_data.iterrows()))

print(f"Fetched data for {len(training_array)} training points")

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
classifier = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=10,
    n_jobs=-1,
    random_state=42,
)

training_data = np.array(training_array)[:, 2:]
classes = np.array(training_array)[:, 1]

model = classifier.fit(training_data, classes)

In [ ]:
# This is for a small subset region
# ll = -17.82845, 177.30492
# ur = -17.49281, 177.77527
# predict_data = load(items, bbox=(ll[1], ll[0], ur[1], ur[0]), measurements=bands, chunks={})

# This one loads all the data from all Viti Levu
predict_data = load(items, bbox=fiji_bbox, measurements=bands, chunks={})

predict_data = predict_data.compute()

predict_data = predict_data.squeeze("time")
predict_data = predict_data.fillna(-9999)
predict_data

In [11]:
# This runs the actual prediction
predicted = predict_xr(model, predict_data, proba=True)

NameError: name 'predict_xr' is not defined

In [12]:
# Convert to int
cleaned_predictions = predicted.copy(deep=True)
cleaned_predictions.Predictions.data = predicted.Predictions.data.astype(np.int8)
cleaned_predictions.Probabilities.data = predicted.Probabilities.data.astype(np.float32)

cleaned_predictions = cleaned_predictions.rename({"Predictions": "lulc", "Probabilities": "prob"})

cleaned_predictions.prob.plot.imshow(size=10)

NameError: name 'predicted' is not defined

In [13]:
# Write GeoTIFF
cleaned_predictions.prob.odc.write_cog("prob_fiji.tif", overwrite=True)

NameError: name 'cleaned_predictions' is not defined